### [ 컨테이너 모듈 ]
- NN의 Module을 여러 개 담아주는 모듈
- 대표 : nn.Sequential, nn.ModuleList, nn.ModuleDict

[1] 모듈 로딩 및 데이터 준비 <hr>

In [76]:
# ================================================================
# [1-1] 모듈 로딩
# ================================================================
import pandas as pd

import torch
import torch.nn as nn

from torchinfo import summary
from collections import OrderedDict

# ================================================================
# [1-2] 데이터 준비
# ================================================================
dataDF = pd.read_csv("../DATA/study_score_multi.csv")
display(dataDF.head())
display(dataDF.info())

,study_hours,sleep_hours,participation,score
0,1.45,5.28,97.4,32.8
1,2.17,5.46,22.4,32.1
2,8.03,5.56,15.6,61.0
3,3.97,4.08,31.2,31.3
4,6.76,7.28,66.8,72.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   study_hours    5000 non-null   float64
 1   sleep_hours    5000 non-null   float64
 2   participation  5000 non-null   float64
 3   score          5000 non-null   float64
dtypes: float64(4)
memory usage: 156.4 KB


None

[2] nn.Sequential 클래스 <hr>

In [77]:
# ================================================================
# [2-1] 기본 형태
# ================================================================
model = nn.Sequential(
    nn.Linear(3, 10),  # (1) 입력층 -> 은닉층1
    nn.ReLU(),         # (2) 활성화 함수 (인스턴스)
    nn.Linear(10, 5),  # (3) 은닉층1 -> 은닉층2
    nn.ReLU(),         # (4) 활성화 함수
    nn.Linear(5, 1)    # (5) 은닉층2 -> 출력층
)


In [78]:
# 모델 구성 요소
print("모델 층 구조", model)

print("\n\n======== 모델 층별 파라미터 ========")
for name, param in model.named_parameters():
    print(f"모델 층별 파라미터 : {name} ->", param.shape)
# "어? 나는 입력 3, 출력 10 (3 -> 10)으로 만들었는데 왜 [10, 3]이지?"
# PyTorch는 내부 연산 최적화를 위해 가중치 행렬을 [출력 개수, 입력 개수] 형태로 저장하고, 
# 계산할 때 뒤집어서(Transpose) 사용하기 때문입니다.

# 자식들만 
print("\n\n======== child-module ========")
for name, child in model.named_children():
    print(f"child : {name} ->", child)

# 자기 자신 + 자식들 + 손자까지 모두
print("\n\n======== module ========")
for module_ in model.modules():
    print(f"module_ : \n", module_)

모델 층 구조 Sequential(
  (0): Linear(in_features=3, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
)


======== 모델 층별 파라미터 ========
모델 층별 파라미터 : 0.weight -> torch.Size([10, 3])
모델 층별 파라미터 : 0.bias -> torch.Size([10])
모델 층별 파라미터 : 2.weight -> torch.Size([5, 10])
모델 층별 파라미터 : 2.bias -> torch.Size([5])
모델 층별 파라미터 : 4.weight -> torch.Size([1, 5])
모델 층별 파라미터 : 4.bias -> torch.Size([1])


======== child-module ========
child : 0 -> Linear(in_features=3, out_features=10, bias=True)
child : 1 -> ReLU()
child : 2 -> Linear(in_features=10, out_features=5, bias=True)
child : 3 -> ReLU()
child : 4 -> Linear(in_features=5, out_features=1, bias=True)


======== module ========
module_ : 
 Sequential(
  (0): Linear(in_features=3, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1

In [79]:
# 순방향
model(torch.tensor([[1, 2, 3]], dtype=torch.float32))

tensor([[-0.5604]], grad_fn=<AddmmBackward0>)

In [80]:
# 모델 요약 정보
summary(model, input_size=(1, 3))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 1]                    --
├─Linear: 1-1                            [1, 10]                   40
├─ReLU: 1-2                              [1, 10]                   --
├─Linear: 1-3                            [1, 5]                    55
├─ReLU: 1-4                              [1, 5]                    --
├─Linear: 1-5                            [1, 1]                    6
Total params: 101
Trainable params: 101
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [82]:
# ================================================================
# [2-2] OrderdDict타입으로 층 구성 : 층에 이름 부여
# ================================================================
# 모델 인스턴스 생성
model = nn.Sequential(
    OrderedDict([
        ("h1", nn.Linear(3, 10)),
        ("relu1", nn.ReLU()),
        ("h2", nn.Linear(10, 5)),
        ("relu2", nn.ReLU()),
        ("out", nn.Linear(5, 1)),
    ])
)

In [83]:
print(model)

Sequential(
  (h1): Linear(in_features=3, out_features=10, bias=True)
  (relu1): ReLU()
  (h2): Linear(in_features=10, out_features=5, bias=True)
  (relu2): ReLU()
  (out): Linear(in_features=5, out_features=1, bias=True)
)


In [84]:
summary(model, input_size=(1, 3))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 1]                    --
├─Linear: 1-1                            [1, 10]                   40
├─ReLU: 1-2                              [1, 10]                   --
├─Linear: 1-3                            [1, 5]                    55
├─ReLU: 1-4                              [1, 5]                    --
├─Linear: 1-5                            [1, 1]                    6
Total params: 101
Trainable params: 101
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00